In [12]:
import requests
import time
import random
from datetime import datetime,timedelta
from enum import Enum

In [13]:
class CircuitState(Enum):
    CLOSED = "closed"
    OPEN = "open"
    HALF_OPEN = "half_open"

In [14]:
class CircuitBreaker:
    def __init__(self, failure_threshold=5, timeout=30, success_threshold=2):
        self.failure_threshold = failure_threshold
        self.timeout = timeout  # seconds to wait before retry
        self.success_threshold = success_threshold 
        self.failure_count = 0
        self.success_count = 0
        self.state = CircuitState.CLOSED
        self.last_failure_time = None
        
    def call(self, func, *args, **kwargs):
        if self.state == CircuitState.OPEN:
            if datetime.now() - self.last_failure_time > timedelta(seconds=self.timeout):
                print("Circuit breaker: Transitioning to HALF_OPEN")
                self.state = CircuitState.HALF_OPEN
            else:
                raise Exception("Circuit breaker is OPEN - request blocked")
        try:
            result = func(*args, **kwargs)
            self._on_success()
            return result
        except Exception as e:
            self._on_failure()
            raise e
    
    def _on_success(self):
        self.failure_count = 0
        if self.state == CircuitState.HALF_OPEN:
            self.success_count += 1
            print(f"Circuit breaker: Success in HALF_OPEN ({self.success_count}/{self.success_threshold})") 
            if self.success_count >= self.success_threshold:
                print("Circuit breaker: Transitioning to CLOSED")
                self.state = CircuitState.CLOSED
                self.success_count = 0
    
    def _on_failure(self):
        self.failure_count += 1
        self.last_failure_time = datetime.now()
        self.success_count = 0
        
        print(f"Circuit breaker: Failure count: {self.failure_count}/{self.failure_threshold}")
        
        if self.failure_count >= self.failure_threshold:
            if self.state != CircuitState.OPEN:
                print("Circuit breaker: Transitioning to OPEN")
                self.state = CircuitState.OPEN

In [15]:
class ResilientClient:
    def __init__(self, base_url, max_retries=3, initial_backoff=1):
        self.base_url = base_url
        self.max_retries = max_retries
        self.initial_backoff = initial_backoff
        self.circuit_breaker = CircuitBreaker()
        self.total_requests = 0
        self.successful_requests = 0
        self.failed_requests = 0
        self.circuit_breaker_blocks = 0
        
    def get_data(self):
        """Get data with retry logic and exponential backoff"""
        self.total_requests += 1
        
        for attempt in range(self.max_retries):
            try:
                result = self.circuit_breaker.call(self._make_request)
                self.successful_requests += 1
                return result
                
            except Exception as e:
                if "Circuit breaker is OPEN" in str(e):
                    print(f"Request blocked by circuit breaker")
                    self.circuit_breaker_blocks += 1
                    self.failed_requests += 1
                    raise e
                
                if attempt < self.max_retries - 1:
                    backoff = self.initial_backoff * (2 ** attempt)
                    jitter = random.uniform(0, 0.3 * backoff)
                    wait_time = backoff + jitter
                    
                    print(f"Attempt {attempt + 1} failed: {e}")
                    print(f"Retrying in {wait_time:.2f}s...")
                    time.sleep(wait_time)
                else:
                    print(f"All {self.max_retries} attempts failed")
                    self.failed_requests += 1
                    raise e
    
    def _make_request(self):
        """Make the actual HTTP request with timeout"""
        response = requests.get(
            f"{self.base_url}/api/first",
            timeout=5 
        )
        response.raise_for_status()
        return response.json()
    
    def print_metrics(self):
        """Print client metrics"""
        success_rate = (self.successful_requests / self.total_requests * 100) if self.total_requests > 0 else 0
        print("\n" + "="*50)
        print("CLIENT METRICS")
        print("="*50)
        print(f"Total Requests:        {self.total_requests}")
        print(f"Successful:            {self.successful_requests}")
        print(f"Failed:                {self.failed_requests}")
        print(f"Circuit Breaker Blocks: {self.circuit_breaker_blocks}")
        print(f"Success Rate:          {success_rate:.1f}%")
        print(f"Circuit Breaker State: {self.circuit_breaker.state.value.upper()}")
        print("="*50 + "\n")

In [16]:
SERVICE_URL = "http://127.0.0.1:59759" 

client = ResilientClient(
        base_url=SERVICE_URL,
        max_retries=3,
        initial_backoff=1
)
    
print("Starting resilient client test...\n")
    
for i in range(20):
    print(f"\n--- Request {i+1} ---")
    try:
        data = client.get_data()
        print(f"Success: {data}")
    except Exception as e:
        print(f"Failure: {e}")
        
    time.sleep(2)

    if (i + 1) % 5 == 0:
        client.print_metrics()

Starting resilient client test...


--- Request 1 ---
Success: {'message': 'Hello World 1'}

--- Request 2 ---
Success: {'message': 'Hello World 1'}

--- Request 3 ---
Success: {'message': 'Hello World 1'}

--- Request 4 ---
Success: {'message': 'Hello World 1'}

--- Request 5 ---
Success: {'message': 'Hello World 1'}

CLIENT METRICS
Total Requests:        5
Successful:            5
Failed:                0
Circuit Breaker Blocks: 0
Success Rate:          100.0%
Circuit Breaker State: CLOSED


--- Request 6 ---
Success: {'message': 'Hello World 1'}

--- Request 7 ---
Success: {'message': 'Hello World 1'}

--- Request 8 ---
Success: {'message': 'Hello World 1'}

--- Request 9 ---
Success: {'message': 'Hello World 1'}

--- Request 10 ---
Success: {'message': 'Hello World 1'}

CLIENT METRICS
Total Requests:        10
Successful:            10
Failed:                0
Circuit Breaker Blocks: 0
Success Rate:          100.0%
Circuit Breaker State: CLOSED


--- Request 11 ---
Success: {'messa